In [16]:
import requests
import json
import time
import pandas as pd
from datetime import datetime

In [12]:
url = "https://api.thegraph.com/subgraphs/name/narutomanchester/wombat-exchange-arb"

currentTime = int(time.time())
checkTime = currentTime - 86400

# create dict to store hour data 
hourData = {}
# loop until scan all second
while checkTime<=currentTime:

  body = """
    query {
      secondDatas(first: 1000, where: { unixTime_gte: %d}) {
        feesUSD
        unixHour
        unixTime
      }
    }
    """%checkTime

  # get request data
  response = requests.post(url, json={"query": body})
  responseData = json.loads(response.text)
  
  # handle request data
  if len(responseData['data']['secondDatas']) == 0:
    break

  for data in responseData['data']['secondDatas']:
    feesUSD, unixHour, unixTime = data['feesUSD'], data['unixHour'], data['unixTime']
    # move to next check time
    checkTime = max(checkTime, unixTime) + 1

    if unixHour not in hourData:
      hourData[unixHour] = 0

    hourData[unixHour] += float(feesUSD)


{1688554800: 0.22256530839999997, 1688558400: 9.630983926104001, 1688562000: 3.0651223319, 1688565600: 4.086869111859318, 1688569200: 1.3456471641000003, 1688572800: 8.6917933914, 1688576400: 0.5065375463, 1688580000: 1.0563067509999993, 1688583600: 1.7477806506532567, 1688587200: 0.489642947, 1688590800: 0.2053399642, 1688594400: 0.2717897681, 1688598000: 5.451302247717571, 1688601600: 2.061511061307299, 1688605200: 1.3778732179, 1688608800: 2.2807737813646476, 1688612400: 2.0830864461479544, 1688616000: 1.0822455532000002, 1688619600: 1.9580035672, 1688623200: 0.8464680645, 1688626800: 0.865124658012473, 1688630400: 1.988348170790894, 1688634000: 7.199913530993883, 1688637600: 0.7061691667000001, 1688641200: 1.1024099998000003}


In [20]:
# Create DF
data = {'date': hourData.keys(),
        'feeUSD': hourData.values()}
  
df = pd.DataFrame(data)
# convert to readable datetime type
df.date = df.date.apply(lambda d: datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d %H'))
print(df)

             date    feeUSD
0   2023-07-05 18  0.222565
1   2023-07-05 19  9.630984
2   2023-07-05 20  3.065122
3   2023-07-05 21  4.086869
4   2023-07-05 22  1.345647
5   2023-07-05 23  8.691793
6   2023-07-06 00  0.506538
7   2023-07-06 01  1.056307
8   2023-07-06 02  1.747781
9   2023-07-06 03  0.489643
10  2023-07-06 04  0.205340
11  2023-07-06 05  0.271790
12  2023-07-06 06  5.451302
13  2023-07-06 07  2.061511
14  2023-07-06 08  1.377873
15  2023-07-06 09  2.280774
16  2023-07-06 10  2.083086
17  2023-07-06 11  1.082246
18  2023-07-06 12  1.958004
19  2023-07-06 13  0.846468
20  2023-07-06 14  0.865125
21  2023-07-06 15  1.988348
22  2023-07-06 16  7.199914
23  2023-07-06 17  0.706169
24  2023-07-06 18  1.102410
